In [20]:
import spacy
from spacy.tokens import DocBin
from tqdm import tqdm
import time
import subprocess
import pandas as pd
import ast
from random import shuffle
import pandas as pd


print("les bibliotheques sont bien importes")

les bibliotheques sont bien importes


In [4]:
print("chargement du modèle spacy pré-entrainé")
nlp_fr = spacy.load("fr_core_news_md")
print("\nmodel spacy bien chargé")

chargement du modèle spacy pré-entrainé

model spacy bien chargé


In [5]:
def test_ner_recognition(text):
    doc = nlp_fr(text)
    spacy.displacy.render(doc, style="ent", jupyter=True)

texte_simple = '''Je souhaiterais me rendre de Paris à Marseille.
Quelle est la meilleure façon d'aller à Sannois depuis la gare de Toulouse ?
Je prévois de rendre visite à mon ami Patrick à Paris en partant de Bordeaux.
Comment puis-je aller de Marseille à Bordeaux sans passer par Paris ?
Quels sont les horaires des trains de Nantes à Lyon demain matin ?
Je dois voyager de Lille à Toulouse ce week-end, avez-vous des suggestions de trajet ?
Quel est le moyen le plus rapide pour me rendre de Lyon à Paris ?'''

test_ner_recognition(texte_simple)

In [21]:
# Chargement du jeu de données à partir d'un fichier CSV pour l'entraînement
print("chargement du jeu de données à partir du dataset csv")
def load_dataset(file_path):
    df = pd.read_csv(file_path, sep=',', encoding='utf-8', on_bad_lines='skip')
    print("\ndataset bien chargé")
    return df

df = load_dataset('../../dataset/raw/initial_training_data.csv')

print("\naperçu du dataset : \n")
display(df) 

chargement du jeu de données à partir du dataset csv

dataset bien chargé

aperçu du dataset : 



,text,entities
0,Je voudrais aller de Toulouse à Bordeaux.,"[{'start': 21, 'end': 29, 'label': 'DEPARTURE'..."
1,Comment me rendre à Port-Boulet depuis la gare...,"[{'start': 20, 'end': 31, 'label': 'DESTINATIO..."
2,Je veux aller voir mon ami Albert à Tours en p...,"[{'start': 36, 'end': 41, 'label': 'DESTINATIO..."
3,Y a-t-il des trains de Nantes à Montaigu ?,"[{'start': 23, 'end': 29, 'label': 'DEPARTURE'..."
4,Je souhaite me rendre à Paris en partant de To...,"[{'start': 25, 'end': 30, 'label': 'DESTINATIO..."
...,...,...
211,Je pars de Lyon pour rejoindre Rennes,"[{'start': 12, 'end': 16, 'label': 'DEPARTURE'..."
212,Je souhaite me rendre à Lyon en partant de Toulon,"[{'start': 25, 'end': 29, 'label': 'DESTINATIO..."
213,Je pars de Lyon pour rejoindre Nîmes,"[{'start': 12, 'end': 16, 'label': 'DEPARTURE'..."
214,Je souhaite me rendre à Lyon en partant de Nice,"[{'start': 25, 'end': 29, 'label': 'DESTINATIO..."


In [9]:
def prepare_data_for_spacy(df):
    print("conversion du dataset csv au format spacy\n")
    db = DocBin()
    data = list(zip(df['text'], df['entities']))
    print("mélangement des données pour éviter l'apprentissage biaisé\n")
    shuffle(data) 

    for text, entities in data:
        entities = ast.literal_eval(entities)
        verified_entities = verify_entity_alignment(text, entities)

        if verified_entities:
            doc = nlp_fr.make_doc(text)
            doc.ents = create_entity_spans(doc, verified_entities)
            db.add(doc)
    
    print("dataset a été bien converti en csv\n")
    return db

In [10]:
def verify_entity_alignment(text, entities):
    print("vérification de l'alignement des entités\n")
    verified_entities = []
    for ent in entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        if text[start:end] == text[start:end]:
            verified_entities.append(ent)
        else:
            print(f"Skipping entity: {text[start:end]} in text: {text}")
            
    print("vérification terminée")        
    return verified_entities

In [13]:
def create_entity_spans(doc, verified_entities):
    print("création de span entités\n")
    ents = []
    for ent in verified_entities:
        start, end, label = ent['start'], ent['end'], ent['label']
        span = doc.char_span(start, end, label=label, alignment_mode="contract")
        if span:
            ents.append(span)
    return ents

# Préparer les données et sauvegarder au format spaCy
db = prepare_data_for_spacy(df)

print("sauvegarde des données au format spaCy")
db.to_disk("../../dataset/processed/processed_training_data.spacy")
print("sauvegardé !")

conversion du dataset csv au format spacy

mélangement des données pour éviter l'apprentissage biaisé

vérification de l'alignement des entités

vérification terminée
création de span entités

vérification de l'alignement des entités

vérification terminée
création de span entités

vérification de l'alignement des entités

vérification terminée
création de span entités

vérification de l'alignement des entités

vérification terminée
création de span entités

vérification de l'alignement des entités

vérification terminée
création de span entités

vérification de l'alignement des entités

vérification terminée
création de span entités

vérification de l'alignement des entités

vérification terminée
création de span entités

vérification de l'alignement des entités

vérification terminée
création de span entités

vérification de l'alignement des entités

vérification terminée
création de span entités

vérification de l'alignement des entités

vérification terminée
création de span entité

In [16]:
def train_model(config_path, output_path, training_data_path):
    print("Génération de la configuration...\n")
    subprocess.run(f"python -m spacy init config {config_path} --lang fr --pipeline ner --optimize efficiency --force", shell=True)
    print("Configuration faite.\n")

    print("Début de l'entraînement du modèle...\n")
    
    # Lancer le processus d'entraînement
    process = subprocess.Popen(f"python -m spacy train {config_path} --output {output_path} --paths.train {training_data_path} --paths.dev {training_data_path}", shell=True, stdout=subprocess.PIPE, stderr=subprocess.PIPE)

    with tqdm(total=100, desc="Entraînement du modèle") as pbar:
        while True:
            # on lit les lignes de sortie pour suivre l'avancement
            output = process.stdout.readline()
            if output == b"" and process.poll() is not None:
                break
            if output:
                print(output.decode().strip()) 

            pbar.update(1)
            time.sleep(1)

    print("Entraînement terminé.\n")

print("Lancement de l'entraînement...")
train_model('./configuration.cfg', '../../models/saved_models', '../../dataset/processed/processed_training_data.spacy')
print("Entraînement terminé.\n")

Lancement de l'entraînement...
Génération de la configuration...

Configuration faite.

Début de l'entraînement du modèle...



Entraînement du modèle:   1%|          | 1/100 [00:04<06:49,  4.13s/it]

ℹ Saving to output directory: ..\..\models\saved_models


Entraînement du modèle:   2%|▏         | 2/100 [00:05<03:44,  2.29s/it]

ℹ Using CPU


...

✔ Saved pipeline to output directory


Entraînement du modèle:  25%|██▌       | 25/100 [06:05<25:45, 20.60s/it]

..\..\models\saved_models\model-last


Entraînement du modèle:  25%|██▌       | 25/100 [06:06<18:18, 14.65s/it]

Entraînement terminé.

Entraînement terminé.



In [19]:
def load_and_test_model(model_path, text):
    print("chargement et test du modèle")
    nlp_model = spacy.load(model_path)
    doc = nlp_model(text)
    return doc

def test_model_on_texts(model_path):
    texts = [
        '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence.
        Je compte me rendre à Bordeaux depuis Marseille pour rendre visite à ma soeur Paris.
        Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert.
        Je dois planifier un voyage Nice Toulouse pour les prochaines vacances.
        Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand la semaine prochaine.''',

        '''Je souhaite me rendre à Lille en partant d'Aubervilliers pour assister à une conférence avec une escale à Nice.
        Je compte me rendre à Bordeaux depuis Marseille en m'arrêtant à Toulouse pour rendre visite à ma soeur Paris.
        Je dois regarder les trains Toulouse - Brest pour aller voir mon ami Albert en faisant une escale à Tours.
        Je dois planifier un voyage Nice Toulouse en passant par Aubervilliers pour les prochaines vacances.
        Une réunion de travail m'oblige à faire Paris - Clermont-Ferrand avec un arrêt par Lyon la semaine prochaine.'''
    ]

    # couleurs pour les entités
    colors = {
        "DEPARTURE": "lightcyan",  
        "DESTINATION": "lightgreen",
        "PERSON": "lightpink",
        "LOC": "lightgoldenrodyellow"
    }

    options = {"ents": list(colors.keys()), "colors": colors}

    for text in texts:
        doc = load_and_test_model("../../models/saved_models/model-best/", text)
        spacy.displacy.render(doc, style="ent", options=options, jupyter=True)

test_model_on_texts("../../models/saved_models/model-best/")

chargement et test du modèle


chargement et test du modèle
